In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
from lib.DrugUtil import flatten, DrugUtil
from IPython.display import display, HTML
import plotly.express as px
from lib.FhirDataUtil import FhirDataUtil
from lib.CCSDataUtil import CCSDataUtil

def printmd(string):
    display(string)

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

_conditions_fhir_file = "input/fhir_conditions_20221003.csv"
_conditions_ccs_file = "input/ccs_conditions.csv"
_medications_fhir_file = "input/fhir_medications_20221003.csv"
_medications_ref_fhir_file = "input/fhir_medications_references_20221003.csv"
_output_dir = 'output/'

_rxnorm_file = "input_static/RxTerms202203/RxTerms202203.txt"
_rxnorm_ingredients_file = "input_static/RxTerms202203/RxTermsIngredients202203.txt"
_rxclass_file = "input_static/_rxclass_2022-04-10.csv"
_ccs_medications_file = "input/ccs_medications.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
fhirUtil = FhirDataUtil()
ccsUtil = CCSDataUtil()
drugUtil = DrugUtil()

drugUtil.load(rxnorm_file=_rxnorm_file,
                        rxnorm_ingredients_file=_rxnorm_ingredients_file, 
                        rxclass_file=_rxclass_file)

fhirUtil.load_conditions(fhir_conditions_file=_conditions_fhir_file, remove_health_concerns=True)

to_numeric_fields = ['diabetes', 'hbp', 'blockages_in_your_coronary', 'heart_attack', 'chf', 'stroke', 'afib', 'sleep_apnea', 'copd', 'asthma', 'cancer', 'immunodeficiency', 'HIV', 'anemia', 'pregnant']
fhirUtil.load_medications(medications_fhir_file=_medications_fhir_file, medications_ref_fhir_file=_medications_ref_fhir_file)
ccsUtil.load_conditions(ccs_conditions_file= _conditions_ccs_file, to_numeric_fields=to_numeric_fields)
ccsUtil.load_medications(ccs_medications_file=_ccs_medications_file)


Reading rxclass file...
Reading rxnorm file...
Loaded FHIR conditions records: 37248
----- Loading Medications ------
Raw FHIR records: 15061
After de-duplicated: 7403
Merge with reference counts:
left_only     5133
both          2236
right_only       0
Name: _merge, dtype: int64
Loaded FHIR medication records 31976  Includes multiple code mappings
Unique users with fhir medications:  1122
------  Done Loading Medications -----


/Users/bubbles/src/python/openepicvsr/lib/CCSDataUtil.py:11: DtypeWarning: Columns (5,6,11,12,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  self.conditions = pd.read_csv(ccs_conditions_file, delimiter=',')


Loaded CCS conditions records: 94735


/Users/bubbles/src/python/openepicvsr/lib/CCSDataUtil.py:17: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  self.medications_raw = pd.read_csv(ccs_medications_file)


# Data Integrity Check: Medications

In [4]:
groups = fhirUtil.medications.drop_duplicates(subset="medication_id").groupby(['provider', '_merge']).size().unstack(fill_value=0)
groups = groups.sort_values(by='left_only', ascending=False)
groups = groups.rename(columns={'left_only':'no_reference', 'both':'yes_reference'})
groups = groups.drop('right_only', axis=1)
# groups.to_csv(_output_dir+'provider_references_tally.csv')
groups


AttributeError: 'FhirDataUtil' object has no attribute 'medications'

In [12]:
ccs = ccsUtil.medications.drop_duplicates('user_id')
fhir = fhirUtil.medications.drop_duplicates('user_id')
m = ccs.merge(fhir, on='user_id', how="outer", indicator=True)
m['_merge'].value_counts()

ValueError: Cannot use name of an existing column for indicator column

# CCS Medications Analysis

In [5]:
ccsUtil.medications

,user_id,site,submitted_at,custom_entry,medication_name,brand_name,full_generic_name,product_name,sxdg_name,rxcui,generic_rxcui,sxdg_rxcui,route,dosage_form,strength,currently_taking,as_needed,frequency_number,frequency_every,frequency_unit,frequency_times_per_unit,variable_basis,reason
0,14,covid,2021-09-27,True,ASPIRIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,not known,NaN
1,14,covid,2021-09-27,False,Atorvastatin (Oral Pill),NaN,atorvastatin 40 MG Oral Tablet,atorvastatin 40 MG Oral Tablet,atorvastatin Pill,617311.0,NaN,1158285.0,Oral Pill,Oral Tablet,40 mg,True,False,1.0,1.0,day,NaN,NaN,NaN
2,14,covid,2021-09-27,False,Dofetilide (Oral Pill),NaN,dofetilide 0.125 MG Oral Capsule,dofetilide 0.125 MG Oral Capsule,dofetilide Pill,310003.0,NaN,1160621.0,Oral Pill,Oral Capsule,0.125 mg,True,False,3.0,12.0,hour,NaN,NaN,NaN
3,14,covid,2021-09-27,False,ELIQUIS (Oral Pill),ELIQUIS,apixaban 5 MG Oral Tablet,apixaban 5 MG Oral Tablet [Eliquis],Eliquis Pill,1364447.0,1364445.0,1364440.0,Oral Pill,Oral Tablet,5 mg,True,False,1.0,12.0,hour,NaN,NaN,Afib
4,14,covid,2021-09-27,False,Ezetimibe (Oral Pill),NaN,ezetimibe 10 MG Oral Tablet,ezetimibe 10 MG Oral Tablet,ezetimibe Pill,349556.0,NaN,1163794.0,Oral Pill,Oral Tablet,10 mg,True,False,1.0,1.0,day,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133456,556213,BA1,2021-12-04,True,Cloidogrel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75mg,True,False,1.0,NaN,day,NaN,NaN,NaN
133457,556213,BA1,2021-12-04,False,Lansoprazole (Oral Disintegrating),NaN,lansoprazole 15 MG Disintegrating Oral Tablet,lansoprazole 15 MG Disintegrating Oral Tablet,lansoprazole Disintegrating Oral Product,351261.0,NaN,1295364.0,Oral Disintegrating,Disintegrating Oral Tablet,15 mg,True,False,2.0,NaN,day,NaN,NaN,NaN
133458,556213,BA1,2021-12-04,False,Levothyroxine (Oral Pill),NaN,levothyroxine sodium 0.075 MG Oral Capsule,levothyroxine sodium 0.075 MG Oral Capsule,levothyroxine Pill,905458.0,NaN,1602745.0,Oral Pill,Oral Capsule,0.075 mg,True,False,1.0,NaN,day,NaN,NaN,NaN
133459,556213,BA1,2021-12-04,False,Losartan (Oral Pill),NaN,losartan potassium 100 MG Oral Tablet,losartan potassium 100 MG Oral Tablet,losartan Pill,979480.0,NaN,1165343.0,Oral Pill,Oral Tablet,100 mg,True,False,1.0,NaN,day,NaN,NaN,NaN


# Medications

In [6]:
fhir_meds = fhirUtil.get_coded_medications(drugUtil)

Unique medications have have a reference:  2236
After removing nan codes:  2236
Unique medications that have a reference & have ingredient code:  2174


In [7]:
ccs_meds = ccsUtil.add_ingredient_columns(drugUtil)
ccs_meds_fhir_users = ccs_meds[ccs_meds['user_id'].isin(fhir_meds['user_id'].values)]

In [8]:
print(f'CCS Medication records: {len(ccsUtil.medications)}')
# ccs_meds = ccsUtil.medications.drop('_merge', axis=1)
fhir_meds_first = fhirUtil.medications.drop('_merge', axis=1)
fhir_meds_first= fhir_meds_first.drop_duplicates(subset='user_id',keep='first')
ccs_meds_first = ccsUtil.medications.drop_duplicates(subset='user_id', keep='first')
ccs_conditions = ccsUtil.conditions.drop_duplicates(subset='user_id', keep='first')
fhir_ccs_merge = ccs_meds_first.merge(fhir_meds_first, on="user_id", how="outer", indicator=True)
print(f'User intersection:')
print(fhir_ccs_merge['_merge'].value_counts())
fhir_users = fhir_ccs_merge.drop_duplicates(subset='user_id')['user_id']
len(fhir_users)

CCS Medication records: 133461
User intersection:
left_only     36550
right_only      599
both            523
Name: _merge, dtype: int64


37672

In [9]:
fhir_ccs_merge[fhir_ccs_merge['_merge'] == 'right_only'].head(100)

,user_id,site,submitted_at,custom_entry,medication_name,brand_name,full_generic_name,product_name,sxdg_name,rxcui,generic_rxcui,sxdg_rxcui,route,dosage_form,strength,currently_taking,as_needed,frequency_number,frequency_every,frequency_unit,frequency_times_per_unit,variable_basis,reason,uploaded_date,provider,...,status,intent,medication_reference,text,authoredon,requester_display,recorder_display,course_of_therapy,dosage_text,dosage_patientinstruction,dosage_as_needed,dosage_route,dosage_method,dose_text,dose_quantity_value,dose_quantity_unit,start,repeats,quantity,quantity_unit,supply,supply_unit,reference_id,coding_code,_merge
37073,502639,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-11,openepic_shared_sutter_health,...,active,order,Medication/esNeiZiMQ2QbYpiTcLFB6NQik0HA08lHs5Z36h7ROG0RDYkvkyPYMLYCodPlLklxEb68EPl45snhsR3BhmQ5.M1dGoJJSHNHWfsOe0ffu-DA3,ondansetron ODT 8mg Orally Disintegrating Tablet,2018-03-09,"Jennifer Papazian Clary, MD","Jennifer Papazian Clary, MD",Short course (acute) therapy,Place 1 Tab on tongue and let dissolve every 12 hours as needed for Nausea,Place 1 Tab on tongue and let dissolve every 12 hours as needed for Nausea,t,Translingual,NaN,ordered,8.0,mg,2018-03-09,3.0,36.0,Tab,18.0,18.0,NaN,NaN,right_only
37074,545431,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-06,openepic_shared_university_of_utah_healthcare,...,active,order,Medication/eDMxVKJnVwMtrnVwH10alvuvVbcl.Es-8SKqKKD4zc1uVoJCZZI0XTTAad3h9oJmoX.u7NAoi2geg54H48LI3RN12rYLMua1cbQJl6RujpqU3,lisinopril 10 mg tablet,2022-06-30,"Mohammed Fahd Qazi, MD","Mohammed Fahd Qazi, MD",NaN,Take 1 (one) tablet (10 mg) by mouth once daily.,Take 1 (one) tablet (10 mg) by mouth once daily.,f,Oral,Take,ordered,10.0,mg,2022-06-30,2.0,90.0,tablet,90.0,90.0,NaN,NaN,right_only
37075,545573,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-06,openepic_shared_university_of_utah_healthcare,...,active,order,Medication/e768yAyUkchg8.LdL6KK0cTtNjP0ChfzBeq1GHD80Z9TpBrUJQqDmX2kk96O8a26YvxV1ZFBs2acBe2qv1MeclpBV5Z-NAUEmmFfzHrt9gXY3,SUMAtriptan 100 mg tablet,2022-06-29T20:51:40Z,NaN,Kelly,NaN,Take 50 mg by mouth every 2 hours as needed for migraine. Do not exceed 200 mg in 24 hours,Take 50 mg by mouth every 2 hours as needed for migraine. Do not exceed 200 mg in 24 hours,t,Oral,Take,ordered,50.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
37076,544931,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-17,openepic_shared_university_of_utah_healthcare,...,active,order,Medication/eNywMj-iMqKbwz912TOzgsnAxadLAqO-8tGOboG75TU5IWOE6qde3jAQZrmuw2PO5bvGaD-RoY7FfgvYdQKiYwZTo-aJcHSN7ZU12j0tACUI3,tamsulosin 0.4 MG capsule,2022-05-03,Jane,Jane,NaN,Take 1-2 capsules (0.4-0.8 mg) by mouth once daily.,Take 1-2 capsules (0.4-0.8 mg) by mouth once daily.,f,Oral,Take,ordered,NaN,NaN,2022-05-03,1.0,60.0,capsule,30.0,30.0,NaN,NaN,right_only
37077,419858,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-27,openepic_shared_ssm_health,...,active,order,Medication/edme8owhxicJYTPH7NaaD8zDqmzQXQDoJE6vq5fs3JWVsYGU9E621AkEr3mSQ2oUOMox-o8q1O5j3TKtcmvGJqpUTRNQ85BeQKJuxYwdkOM83,levonorgestrel 20 MCG/DAY IUD,2017-08-01T18:09:21Z,NaN,Medical Assistant Kathryn,Short course (acute) therapy,1 Device by Intrauterine route as directed,1 Device by Intrauterine route as directed,f,Intrauterine,NaN,ordered,1.0,device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only
37078,461353,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-22,openepic_shared_penn_medicine,...,active,order,Medication/edZbxUXEOLRk0R6i.Rl6Wj0fCvdnziMmbVyDfukBFp6zNwpcVVbNlMs0AWDhN0m73EzSzfZhV7k1x1QD.ze2BnWapXSz1YsriOGd2BLxqJaE3,Chondroitin Sulfate 400 MG Caps,2022-04-19T13:37:42Z,NaN,Tara Grogan,Short course (acute) therapy,"Take 1,200 mg by mouth daily.","Take 1,200 mg by m

In [10]:
insulin_codes = [ 400008,  253182,  253182,  253182,  253182,  253182,  274783,
        274783,  139825,  253182,  253182,  274783,  253182,  253182,
        400008,  400008,   86009,   86009,   86009,  314684,   86009,
         86009, 1605101,   86009, 1670007,  274783, 1605101, 1605101,
        253182,  253182,  253182,  253182, 1670007,   51428,  274783,
        253182, 1670007,   86009,   86009,  274783,  274783,  274783,
        274783,  400008,  274783, 1605101,  253182,  274783,  139825,
         86009,   51428, 1605101,  253182,   86009,  274783,   86009,
       1670007, 1605101,  253182,  253182,  253182,  253182,   86009,
       1670007,   51428, 1605101,  253182,   51428,  253182,  253182,
        314684,   86009,  253182,   86009, 1605101,   51428,  352385,
         86009,   51428,  352385,   86009,  314684,   86009,   86009,
         86009,  314684,   86009,  139825,  139825,  274783,  274783,
        314684,   86009,  314684,   86009, 1605101,  352385,   86009,
       1670007, 1670007, 1605101,   51428,  253182,  253182,  274783,
        274783,  274783,   86009,  314684,   86009,   86009,   86009,
         51428,  274783,   86009,   86009,   86009,  274783,  274783,
        253182, 1605101,  352385,   51428,   86009,   51428,  314684,
         51428,   86009, 1605101,  253182,   51428,   51428, 1670007,
        274783,  253182]
statin_codes = [83367, 301542, 41127, 42463, 36567]
depression_codes = [42347, 36437, 321988, 4493, 39786, 72625, 10737, 2418, 32937]
allergy_codes = [41126, 20610, 28889, 88249, 87636, 356887, 5553]
hypertension_codes = [29046, 52175, 17767, 5487, 9997, 69749]
ocp_codes = [4124, 6373, 8727, 14584, 7514]
sleep_codes = [6711, 10737, 39993, 704]
anxiety_codes = [596, 2598, 6470]
anticoagulation_codes = [1364430, 11289, 1114195, 1599538, 5224, 67109, 67108]
vitamins = [2418, 278, 11246, 11256, 11248, 42954,  8308, 11258, 42781, 11251]

In [53]:
def compareDatasets(codes, fhir_meds, ccs_meds):
    result_fhir = fhir_meds[fhir_meds['coding_code'].isin(codes)].drop_duplicates(subset='user_id')
    result_ccs = ccs_meds[ccs_meds['ING_RXCUI'].isin(codes)].drop_duplicates(subset='user_id')
    print(f'FHIR records that match codes: {len(result_fhir)}')
    print(f'CCS records that match codes: {len(result_ccs)}')
    result_fhir = result_fhir.drop('_merge', axis=1)
    outer_merge = result_ccs.merge(result_fhir, on='user_id', how="outer", indicator=True)
    return outer_merge

outer_merge = compareDatasets(insulin_codes, fhir_meds, ccs_meds_fhir_users)
print('\nCCS left-merge to FHIR:')
counts = outer_merge['_merge'].value_counts(dropna=False).reset_index(name="counts")
counts['index'] = counts['index'].str.replace('right_only','FHIR Only')
counts['index'] = counts['index'].str.replace('left_only','CCS Only')
counts

FHIR records that match codes: 7
CCS records that match codes: 6

CCS left-merge to FHIR:


,index,counts
0,FHIR Only,4
1,CCS Only,3
2,both,3


In [42]:
ccs_meds_fhir_users[ccs_meds_fhir_users['ING_RXCUI'].isin(insulin_codes)].drop_duplicates(subset='user_id')

,user_id,site,submitted_at,custom_entry,medication_name,brand_name,full_generic_name,product_name,sxdg_name,rxcui,generic_rxcui,sxdg_rxcui,route,dosage_form,strength,currently_taking,as_needed,frequency_number,frequency_every,frequency_unit,frequency_times_per_unit,variable_basis,reason,RXCUI,INGREDIENT,ING_RXCUI
8630,370815,CCS11,2021-11-13,False,LANTUS (Injectable),LANTUS,3 ML insulin glargine 100 UNT/ML Pen Injector,3 ML insulin glargine 100 UNT/ML Pen Injector [Lantus],Lantus Injectable Product,847232.0,847230.0,1175624.0,Injectable,Pen Injector,100 unt/ml,True,False,45.0,1.0,day,NaN,NaN,NaN,847232.0,insulin glargine,274783.0
39539,392160,covid19,2021-12-07,False,LANTUS (Injectable),LANTUS,3 ML insulin glargine 100 UNT/ML Pen Injector,3 ML insulin glargine 100 UNT/ML Pen Injector [Lantus],Lantus Injectable Product,847232.0,847230.0,1175624.0,Injectable,Pen Injector,100 unt/ml,True,False,1.0,1.0,day,NaN,NaN,Diabetes,847232.0,insulin glargine,274783.0
65783,416365,bethematch,2021-11-30,False,"Insulin analog, lispro (Injectable)",NaN,3 ML insulin lispro 100 UNT/ML Pen Injector,3 ML insulin lispro 100 UNT/ML Pen Injector,insulin lispro Injectable Product,1652639.0,NaN,1157461.0,Injectable,Pen Injector,100 unt/ml,True,True,0.0,0.0,hour,NaN,NaN,NaN,1652639.0,insulin lispro,86009.0
119173,490529,hopeccs,2021-10-15,False,NovoLOG (Injectable),NOVOLOG,"insulin aspart, human 100 UNT/ML Injectable Solution","insulin aspart, human 100 UNT/ML Injectable Solution [NovoLog]",NovoLog Injectable Product,351926.0,311040.0,1178119.0,Injectable,Injectable Solution,100 unt/ml,True,True,0.0,0.0,hour,NaN,NaN,Type 1 diabetes,351926.0,"insulin aspart, human",51428.0
130717,524349,BA1,2021-11-27,False,LANTUS (Injectable),LANTUS,3 ML insulin glargine 100 UNT/ML Pen Injector,3 ML insulin glargine 100 UNT/ML Pen Injector [Lantus],Lantus Injectable Product,847232.0,847230.0,1175624.0,Injectable,Pen Injector,100 unt/ml,True,False,0.0,0.0,day,NaN,NaN,NaN,847232.0,insulin glargine,274783.0
139839,553628,ny1,2021-12-06,False,NovoLOG (Injectable),NOVOLOG,"insulin aspart, human 100 UNT/ML Injectable Solution","insulin aspart, human 100 UNT/ML Injectable Solution [NovoLog]",NovoLog Injectable Product,351926.0,311040.0,1178119.0,Injectable,Injectable Solution,100 unt/ml,True,True,0.0,0.0,hour,NaN,NaN,NaN,351926.0,"insulin aspart, human",51428.0


In [55]:

df2 = ccsUtil.medications_raw.merge(ccsUtil.medications_raw.groupby('user_id')['submitted_at'].max().reset_index(),
                            on=['user_id', 'submitted_at'], how='inner')

df2 = df2.dropna(subset='full_generic_name')

df2 = df2[df2['full_generic_name'].str.contains('insulin')].drop_duplicates(subset='user_id')
len(df2)
# df1 = fhir_meds.drop_duplicates(subset='user_id')
# mer = df2.merge(df1, on="user_id", how="inner")
# len(mer)

104738

In [13]:
top_meds = ccs_meds['INGREDIENT'].value_counts(dropna=False).reset_index().head(100)
top_meds.rename(columns={'INGREDIENT': 'count'}, inplace=True)
top_meds.rename(columns={'index': 'ingredient'}, inplace=True)
def getIngredient(q):
    try:
        return drugUtil.findIngredientNameL(q, 0).iloc[0]['ING_RXCUI']
    except: return ''
top_meds['code'] = top_meds.apply(lambda r: getIngredient(r['ingredient']), axis=1)
top_meds

,ingredient,count,code
0,NaN,29483,
1,levothyroxine,3829,10582
2,atorvastatin,3104,83367
3,fluticasone,2910,41126
4,bupropion,2773,42347
5,cetirizine,2732,20610
6,sertraline,2467,36437
7,lisinopril,2079,29046
8,escitalopram,2070,321988
9,omeprazole,1954,7646


In [14]:
drugUtil.findIngredientNameL('enoxaparin', maxdistance=0)#.drop_duplicates(subset='ING_RXCUI')['ING_RXCUI'].values)#['ING_RXCUI'].values
      

,RXCUI,INGREDIENT,ING_RXCUI,match
25085,854236,enoxaparin,67108,0
25082,854228,enoxaparin,67108,0
25083,854232,enoxaparin,67108,0
25084,854235,enoxaparin,67108,0
25086,854238,enoxaparin,67108,0
25087,854239,enoxaparin,67108,0
25088,854241,enoxaparin,67108,0
25090,854245,enoxaparin,67108,0
25089,854242,enoxaparin,67108,0
25092,854248,enoxaparin,67108,0


In [174]:
fhir_meds[fhir_meds['coding_code'].isin(insulin_codes)]['user_id']

586      370815
2973     392160
12509    517050
15445    426576
20757    524349
29498    234461
30215    490529
30662    490529
30998    358704
Name: user_id, dtype: int64

In [171]:
fhirUtil.medications[fhirUtil.medications.text.str.contains('insulin')]
fhirUtil.medications[fhirUtil.medications['coding_code'] == 274783]
# fhirUtil.medications_ref[fhirUtil.medications_ref['id'] == 'e2mh2O4CE7ucP0QD6xNd6IJuUgMVa6QKMhPZY9yRWQegf1J3aHe5XRNKT9bnZx2r4ArD2MwV.KkVE0f5f4q5v-97hssJdfKHPiiw.gXrt2T43']


,user_id,uploaded_date,provider,medication_id,status,intent,medication_reference,text,authoredon,requester_display,recorder_display,course_of_therapy,dosage_text,dosage_patientinstruction,dosage_as_needed,dosage_route,dosage_method,dose_text,dose_quantity_value,dose_quantity_unit,start,repeats,quantity,quantity_unit,supply,supply_unit,reference_id,coding_code,_merge
586,370815,2022-08-11,openepic_shared_ucsf_health,eo29OH4orx2F3GBYA637JJAyY6Nb5wcx8s0UvuDUjYhM3,active,order,Medication/eBVJt45GILH.ouIp6eLcREaKAvfG2rOu.9eplJz3YzNkEvsGAF8qKMx.2uivxcSWRjptNIxqrQFFIUvj.e2lPFU.CcYp-H14PMuVjtLW56SM3,insulin glargine 100 unit/mL injection,2020-05-22T20:26:27Z,NaN,Alissa C,Short course (acute) therapy,Inject 43 Units under the skin nightly at bedtime,Inject 43 Units under the skin nightly at bedtime,f,Subcutaneous,Inject,ordered,43.0,Units,NaN,NaN,NaN,NaN,NaN,NaN,eBVJt45GILH.ouIp6eLcREaKAvfG2rOu.9eplJz3YzNkEvsGAF8qKMx.2uivxcSWRjptNIxqrQFFIUvj.e2lPFU.CcYp-H14PMuVjtLW56SM3,274783.0,both
2973,392160,2022-08-17,openepic_shared_wellspan_health,eCvGjYZofgtbG1fRaZoCWNmXS2VDcLpHpc2qQ-yMngTg3,active,order,Medication/eXJ0rf4Nss0Fh157Xa64-gVyMTpHMN3g2OzHll1SURSueFp4vUFOS9FyEG72LKz6eftZFCNNrQSRnlI43-.mkQd68MfrOPjqp-ShsS8ZxW703,LANTUS SOLOSTAR U-100 INSULIN 100 unit/mL (3 mL) insulin pen,2021-03-22,"Bridget A Hilliard, MD",Tai M,Continuous long term therapy,INJECT 10 UNITS UNDER THE SKIN NIGHTLY,INJECT 10 UNITS UNDER THE SKIN NIGHTLY,f,subcutaneous,NaN,ordered,10.0,Units,2021-03-22,3.0,45.0,mL,NaN,NaN,eXJ0rf4Nss0Fh157Xa64-gVyMTpHMN3g2OzHll1SURSueFp4vUFOS9FyEG72LKz6eftZFCNNrQSRnlI43-.mkQd68MfrOPjqp-ShsS8ZxW703,274783.0,both
20757,524349,2022-08-19,openepic_shared_houston_methodist,eh6VIdO5w9gsw4KtiDyS4fCEtbu9JGP6pPBgfJpZ2hIs3,active,order,Medication/e2mh2O4CE7ucP0QD6xNd6IJuUgMVa6QKMhPZY9yRWQegf1J3aHe5XRNKT9bnZx2r4ArD2MwV.KkVE0f5f4q5v-97hssJdfKHPiiw.gXrt2T43,Lantus Solostar U-100 Insulin 100 unit/mL injection (pen),2022-03-24,"Ali Majid Sawal, DO","Genette Llorens, MA",Short course (acute) therapy,"Inject 0.2 mL (20 Units total) under the skin nightly., Starting Thu 3/24/2022, Normal",Inject 0.2 mL (20 Units total) under the skin nightly.,f,subcutaneous,Inject,ordered,20.0,Units,2022-03-24,3.0,6.0,pen,90.0,90.0,e2mh2O4CE7ucP0QD6xNd6IJuUgMVa6QKMhPZY9yRWQegf1J3aHe5XRNKT9bnZx2r4ArD2MwV.KkVE0f5f4q5v-97hssJdfKHPiiw.gXrt2T43,274783.0,both
30662,490529,2022-08-29,openepic_shared_stanford_health_care,eaXqerNiDXjD.tXuiwDQvrrMYSGh4IDZO6rn3N11R6kQ3,active,order,Medication/eTs96xYJwGDWibPZK1tq7vlRUj5f7WVZbetrychIjeosDcOPkjR5WUEMx.52khQ0X.LBBhS2KvS8uTaOfItkqnmbKsPglV2ZkPPBw0v.Jpz03,Lantus Solostar U-100 Insulin 100 unit/mL (3 mL) Inpn,2022-03-29,"Marina Basina, MD","Marina Basina, MD",Short course (acute) therapy,inject 40 Units subcutaneous (under the skin) every day,inject 40 Units subcutaneous (under the skin) every day,f,Subcutaneous,Inject,ordered,40.0,Units,2022-03-29,4.0,2.0,Syringe,NaN,NaN,eTs96xYJwGDWibPZK1tq7vlRUj5f7WVZbetrychIjeosDcOPkjR5WUEMx.52khQ0X.LBBhS2KvS8uTaOfItkqnmbKsPglV2ZkPPBw0v.Jpz03,274783.0,both
30998,358704,2022-08-11,openepic_shared_ssm_health,ekVPzAYAo01BL2mOLdLNwQ2OL6EnRhRXc8dlZ3wTLXFI3,active,order,Medication/euOcosw1DlgPsL3Zhb1KDvbJxTZP64TtJ4f1hEoWc64VAguVhusOmGHAuQhmYzp2O.KNkdzj7yHgkQu3PUwMrporuyxKQQRR-dmuXl3YXMlg3,Basaglar KwikPen pen,2021-04-12,"Suresh Chode, MD","Suresh Chode, MD",Short course (acute) therapy,Inject 70 (seventy) Units subcutaneously at bedtime,Inject 70 (seventy) Units subcutaneously at bedtime,f,Subcutaneous,Inject,ordered,70.0,Units,2021-04-12,0.0,15.0,mL,21.0,21.0,euOcosw1DlgPsL3Zhb1KDvbJxTZP64TtJ4f1hEoWc64VAguVhusOmGHAuQhmYzp2O.KNkdzj7yHgkQu3PUwMrporuyxKQQRR-dmuXl3YXMlg3,274783.0,both


In [178]:
fhirUtil.medications[fhirUtil.medications['user_id'] == 370815].drop_duplicates(subset='medication_id')

,user_id,uploaded_date,provider,medication_id,status,intent,medication_reference,text,authoredon,requester_display,recorder_display,course_of_therapy,dosage_text,dosage_patientinstruction,dosage_as_needed,dosage_route,dosage_method,dose_text,dose_quantity_value,dose_quantity_unit,start,repeats,quantity,quantity_unit,supply,supply_unit,reference_id,coding_code,_merge
345,370815,2022-08-11,openepic_shared_ucsf_health,e7SfaAZwLtyhFfSybrkgyZpywSiBplo-yZU-cTy5E1HY3,active,order,Medication/eytbKeNNEGSt6.OLKrXfE24VSEqMx2R3ne3AuRTpiexpNZ-GzvtuxYR3bYrsfEdHAJbxzumSkl0VzaIogwulOpUwNADVV5e5896ub6A.voEU3,omeprazole 40 mg capsule,2014-10-02T22:26:30Z,NaN,Caitlin J,Short course (acute) therapy,Take 40 mg by mouth Daily.,Take 40 mg by mouth Daily.,f,Oral,NaN,ordered,40.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eytbKeNNEGSt6.OLKrXfE24VSEqMx2R3ne3AuRTpiexpNZ-GzvtuxYR3bYrsfEdHAJbxzumSkl0VzaIogwulOpUwNADVV5e5896ub6A.voEU3,151993.0,both
353,370815,2022-08-11,openepic_shared_ucsf_health,ePj2qBeGtdM45ExtG69VBkB.tiGWtRomkVJ.mWV2DhK43,active,order,Medication/eJgCDpQ-DSuBU6RvyljJqQ0bcbuZVcDX7FLlo72dPmgwivulOJWrF0GqFPjhSMLD-XUbxyLBIw0XCdwKOqktQe-cWpCzPRL7w6QaauUBNnX43,ranitidine 150 mg tablet,2016-01-14T23:38:51Z,NaN,Huimei Z,Short course (acute) therapy,Take 150 mg by mouth 2 (two) times daily.,Take 150 mg by mouth 2 (two) times daily.,f,Oral,Take,ordered,150.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eJgCDpQ-DSuBU6RvyljJqQ0bcbuZVcDX7FLlo72dPmgwivulOJWrF0GqFPjhSMLD-XUbxyLBIw0XCdwKOqktQe-cWpCzPRL7w6QaauUBNnX43,198191.0,both
367,370815,2022-08-11,openepic_shared_ucsf_health,eS2Zyjxvl8insUnQUZZdPmPvcwQyqOwrDTjobvFi7lks3,active,order,Medication/enczjTtHT80Mcdzoc17bWa4M3l.quJXIrkMnrtc8a7HfxqaLGil.FGT8C0o5s3u7vkJgtcdIQZ2CWm6-oHHopJtdk2xU7v7Pkj81YIx7gOjI3,amLODIPine 5 mg tablet,2020-05-22T20:26:27Z,NaN,Alissa C,Short course (acute) therapy,Take 5 mg by mouth daily,Take 5 mg by mouth daily,f,Oral,Take,ordered,5.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,enczjTtHT80Mcdzoc17bWa4M3l.quJXIrkMnrtc8a7HfxqaLGil.FGT8C0o5s3u7vkJgtcdIQZ2CWm6-oHHopJtdk2xU7v7Pkj81YIx7gOjI3,2599174.0,both
490,370815,2022-08-11,openepic_shared_ucsf_health,eSVMdmFOsncUFaPHbmTgA64hbvMxSNKtrF9a-7DhAV4Y3,active,order,Medication/eMiIlkBRFRFrjI1yaiaxthg9lIYHD.87StykxVQzEI69.sf4hw6G1V-FpMN4WGuC9YF9g6zxNWIBpyp8Oa9crPkqwt-2fUGNSW--oLvdzeuA3,glipiZIDE 5 mg tablet,2016-01-14T23:38:51Z,NaN,Huimei Z,Short course (acute) therapy,Take 5 mg by mouth 2 (two) times daily.,Take 5 mg by mouth 2 (two) times daily.,f,Oral,Take,ordered,5.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eMiIlkBRFRFrjI1yaiaxthg9lIYHD.87StykxVQzEI69.sf4hw6G1V-FpMN4WGuC9YF9g6zxNWIBpyp8Oa9crPkqwt-2fUGNSW--oLvdzeuA3,201921.0,both
498,370815,2022-08-11,openepic_shared_ucsf_health,eUlkIMllCZyiPvjRnof765JGY87U2hoIYrWTcEwMJ3MA3,active,order,Medication/eZ6PM4lwAis9UvXUz3wPCzwGmHbuFK1VMa0ksCVDrYRiZDIsd-T42jRAIUZRLB8o-01F57q-qaaktDeQntol--B9C5UCWu8IpW-c8ipJkhoc3,simvastatin 10 mg tablet,2014-10-02T22:26:29Z,NaN,Caitlin J,Short course (acute) therapy,Take 10 mg by mouth Daily.,Take 10 mg by mouth Daily.,f,Oral,NaN,ordered,10.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eZ6PM4lwAis9UvXUz3wPCzwGmHbuFK1VMa0ksCVDrYRiZDIsd-T42jRAIUZRLB8o-01F57q-qaaktDeQntol--B9C5UCWu8IpW-c8ipJkhoc3,1944257.0,both
503,370815,2022-08-11,openepic_shared_ucsf_health,eXhwUOXB0l48Hmm80PCUlEvJR8COtXpR3Tp.UHnHjYoE3,active,order,Medication/eHwvzX6ooMTCLJ9xcdAc8lYI1YqcNgmvVMkhG2hCf7KKNXa8zdE9xa0zRNZxTU9MjOxnZM7FlGIo16aC2M3dx0HQI2ODq.VrZe6Z2VWIpswo3,lisinopriL 20 mg tablet,2012-10-23T21:18:42Z,NaN,Belinda P,Short course (acute) therapy,Take 20 mg by mouth Daily.,Take 20 mg by mouth Daily.,f,Oral,NaN,ordered,20.0,mg,NaN,NaN,NaN,NaN,NaN,NaN,eHwvzX6ooMTCLJ9xcdAc8lYI1YqcNgmvVMkhG2hCf7KKNXa8zdE9xa0zRNZxTU9MjOxnZM7FlGIo16aC2M3dx0HQI2ODq.VrZe6Z2VWIpswo3,29046.0,both
512,370815,2022-08-11,openepic_shared_ucsf_health,eZ-QO105rYcSK8x0CW2yqTVgUaMocVwz3W5F4m-bEXKs3,active,order,Medication/e3lqtFJpVeTuaeBDBSUFXGkctL9rWZGXttTMoldk.fFDDnIWB2wFXMzbJfDpkx389YWwvkCq3g2AJWL9w35sEoLcrop.8me4nVM5CiBRS18Q3,atorvastatin 20 mg tablet,2016-01-14T23:38:50Z,NaN,Huimei Z,Short c

In [207]:
cond = fhirUtil.conditions[fhirUtil.conditions['coding_system'] == 'ICD10']
v = cond['coding_code'].value_counts(dropna=False).head(100)
v = v.reset_index().rename(columns={'index': 'icd10', 'coding_code': 'counts'})
c = cond[['condition_text', 'coding_code']].drop_duplicates(subset='coding_code', keep='first')
z = v.merge(c, how='left', right_on="coding_code", left_on='icd10')
z = z.drop('coding_code', axis=1)
# z.to_csv(_output_dir + '/fhir_icd10.csv')
# z = v
z

,icd10,counts,condition_text
0,I10,347,Essential hypertension
1,E78.5,230,Borderline hyperlipidemia
2,Z00.00,191,Healthcare maintenance
3,K21.9,168,Gastroesophageal reflux disease without esophagitis
4,G89.29,143,Chronic low back pain
5,J30.9,129,Allergic rhinitis
6,G47.33,123,OSA (obstructive sleep apnea)
7,F41.9,119,Anxiety
8,E03.9,116,Hypothyroidism
9,E66.9,116,Obesity


In [206]:
def truncate(s):
    return (s[:30] + '..') if len(s) > 30 else s
z['condition_text'] = z['condition_text'].apply(lambda r: truncate(r))
fig = px.bar(z.head(20), x='counts', y='condition_text', orientation='h')
fig.update_yaxes(autorange="reversed", title="ICD10 Codes")
fig.update_xaxes(title="Counts")
fig.update_layout(showlegend=False)
fig.show()